In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("DateFormatting")
    .master("local[*]")
    .getOrCreate()
)

spark


In [4]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

df = spark.createDataFrame(data = emp_data, schema = emp_schema)

df.show(5)

df.printSchema()

+-----------+-------------+----------+---+------+------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|
+-----------+-------------+----------+---+------+------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|
+-----------+-------------+----------+---+------+------+----------+
only showing top 5 rows

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [17]:
# Case When
# select employee_id, name, age, salary, gender,
# case when gender = 'Male' then 'M' when gender = 'Female' then 'F' else null end as new_gender, hire_date from emp
from pyspark.sql.functions import expr

#using case expression directly.
df.withColumn("new_gender", expr("case when gender = 'Male' then 'M' when gender = 'Female' then 'F' else null end")).show(5)


#using when statement.
from pyspark.sql.functions import when, col
df.withColumn("new_gender", when(col("gender") == 'Male', 'M').when(col('gender') == 'Female', 'M').otherwise(None)).show(5)

#df[]
from pyspark.sql.functions import when, col
df.withColumn("new_gender", when(df['gender'] == 'Male', 'M').when(col('gender') == 'Female', 'M').otherwise(None)).show(5)

+-----------+-------------+----------+---+------+------+----------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+----------+---+------+------+----------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|         M|
+-----------+-------------+----------+---+------+------+----------+----------+
only showing top 5 rows

+-----------+-------------+----------+---+------+------+----------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+----------+---+------+------+----------+----------+
|        001|          101|

In [12]:
df.select('gender').show(5)

+------+
|gender|
+------+
|  Male|
|Female|
|  Male|
|Female|
|  Male|
+------+
only showing top 5 rows



In [23]:
# Replace in Strings
# select employee_id, name, replace(name, 'J', 'Z') as new_name, age, salary, gender, new_gender, hire_date from emp_gender_fixed

from pyspark.sql.functions import regexp_replace

df.select("employee_id", "department_id", "name", regex_replace("name", 'J', 'Z') as "new_name" , "age", "gender", "salary", "hire_date").show(5)

SyntaxError: invalid syntax (1722645323.py, line 6)

In [25]:
# Replace in Strings
# select employee_id, name, replace(name, 'J', 'Z') as new_name, age, salary, gender, new_gender, hire_date from emp_gender_fixed

from pyspark.sql.functions import regexp_replace
df.withColumn("new_name", regexp_replace("name", 'J', 'Z')).show(5)

+-----------+-------------+----------+---+------+------+----------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|  new_name|
+-----------+-------------+----------+---+------+------+----------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|  Zohn Doe|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|Zane Smith|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01| Bob Brown|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30| Alice Lee|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01| Zack Chan|
+-----------+-------------+----------+---+------+------+----------+----------+
only showing top 5 rows



In [32]:
# Convert Date
# select *,  to_date(hire_date, 'YYYY-MM-DD') as hire_date from emp_name_fixed

from pyspark.sql.functions import to_date
df.withColumn("hire_date", to_date(col("hire_date"), "yyyy-MM-dd")).show(5)



+-----------+-------------+----------+---+------+------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|
+-----------+-------------+----------+---+------+------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|
+-----------+-------------+----------+---+------+------+----------+
only showing top 5 rows



In [34]:
# Add Date Columns
# Add current_date, current_timestamp, extract year from hire_date

from pyspark.sql.functions import current_date, current_timestamp

df.withColumn("current_date", current_date()).withColumn("current_timestamp", current_timestamp()).show(5, truncate = False)


+-----------+-------------+----------+---+------+------+----------+------------+--------------------------+
|employee_id|department_id|name      |age|gender|salary|hire_date |current_date|current_timestamp         |
+-----------+-------------+----------+---+------+------+----------+------------+--------------------------+
|001        |101          |John Doe  |30 |Male  |50000 |2015-01-01|2025-10-16  |2025-10-16 08:43:36.297368|
|002        |101          |Jane Smith|25 |Female|45000 |2016-02-15|2025-10-16  |2025-10-16 08:43:36.297368|
|003        |102          |Bob Brown |35 |Male  |55000 |2014-05-01|2025-10-16  |2025-10-16 08:43:36.297368|
|004        |102          |Alice Lee |28 |Female|48000 |2017-09-30|2025-10-16  |2025-10-16 08:43:36.297368|
|005        |103          |Jack Chan |40 |Male  |60000 |2013-04-01|2025-10-16  |2025-10-16 08:43:36.297368|
+-----------+-------------+----------+---+------+------+----------+------------+--------------------------+
only showing top 5 rows



In [ ]:
# Drop Null gender records
emp_1 = emp_dated.na.drop()

In [48]:
# Fix Null values
# select *, nvl('new_gender', 'O') as new_gender from emp_dated

from pyspark.sql.functions import coalesce, lit

df.show()

df.withColumn("gender", coalesce(col("gender"), lit("OTHER"))).show()


# ** this is not working, check again. **
# becase, it is blank, not None!

df.withColumn("new_gender", when(col("gender") == 'Male', 'M').when(col("gender") == 'Female', 'F').otherwise(None)).withColumn("new_gender", coalesce(col("new_gender"), lit("O"))).show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [43]:
# Bonus TIP
# Convert date into String and extract date information

from pyspark.sql.functions import date_format

df.withColumn("string_formatted_date", date_format(col("hire_date"), "yyyyMMdd")).show(5)


+-----------+-------------+----------+---+------+------+----------+---------------------+
|employee_id|department_id|      name|age|gender|salary| hire_date|string_formatted_date|
+-----------+-------------+----------+---+------+------+----------+---------------------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|             20150101|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|             20160215|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|             20140501|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|             20170930|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|             20130401|
+-----------+-------------+----------+---+------+------+----------+---------------------+
only showing top 5 rows

